In [ ]:
!pip install -q openai

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 4: Tool Definition

In this exercise, you'll define a tool and see how the model asks to use it.

By the end, you'll understand:
- How to define a tool using OpenAI's function calling format
- That the model doesn't **call** tools — it returns `tool_calls` **asking you** to call them
- The structure of a tool call response

## Step 1: Setup

In [ ]:
import json
import openai

client = openai.OpenAI()

## Step 2: Define a tool

OpenAI tools are wrapped in a `type: "function"` object. Each function has:
- `name` — a unique identifier
- `description` — tells the model when and how to use it
- `parameters` — a JSON Schema describing the expected parameters

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

## Step 3: Send a message that should trigger tool use

When we ask about the weather, the model should recognize that it needs to use the `get_weather` tool.

In [ ]:
messages = [{"role": "user", "content": "What is the weather in Tokyo?"}]

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages,
    tools=tools,
)

## Step 4: Inspect the response

Notice that `finish_reason` is `"tool_calls"`, not `"stop"`. The model is **pausing** and asking us to run a tool.

In [ ]:
print(json.dumps(json.loads(response.to_json()), indent=2))

Look at `message.tool_calls` — each tool call has:
- `id` — a unique ID for this tool call (you'll need this later)
- `function.name` — which tool the model wants to use
- `function.arguments` — the arguments as a **JSON string** (you need to parse it)

In [ ]:
message = response.choices[0].message
for tool_call in message.tool_calls:
    print(f"Tool: {tool_call.function.name}")
    print(f"  Arguments: {tool_call.function.arguments}")
    print(f"  Parsed: {json.loads(tool_call.function.arguments)}")
    print(f"  ID: {tool_call.id}")

## Key takeaway

The model never actually calls your tool. It tells you **which** tool to call and **what arguments** to pass. It's your job to execute the tool and send the result back. That's what the next exercise covers.